In [1]:
import os

In [2]:
!mkdir /content/dataset/

In [3]:
%cd /content/drive/MyDrive/CS5246Project/stock_dataset/
!cp Combined_train.csv /content/dataset/
!cp Combined_valid.csv /content/dataset/
!cp Combined_test.csv /content/dataset/

/content/drive/MyDrive/CS5246Project/stock_dataset


In [ ]:
!pip install datasets
!pip install transformers

In [5]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import string
import re
import time
import torch

In [6]:
import json
import os
from pathlib import Path
from typing import *

import pandas as pd 
import scipy
from datasets import ClassLabel, Dataset
from sklearn import metrics
from sklearn.model_selection import train_test_split
import transformers
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    EarlyStoppingCallback,
    Trainer,
    TrainingArguments,
)
print(transformers.__version__)

4.27.4


In [7]:
df_train = pd.read_csv('/content/dataset/Combined_train.csv')
df_valid = pd.read_csv('/content/dataset/Combined_valid.csv')
df_test = pd.read_csv('/content/dataset/Combined_test.csv')

In [8]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", 
                       "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", 
                       "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", 
                       "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am",
                       "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", 
                       "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have",
                       "it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not",
                       "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", 
                       "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                       "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", 
                       "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have",
                       "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is",
                       "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would",
                       "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have",
                       "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have",
                       "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", 
                       "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did",
                       "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", 
                       "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", 
                       "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                       "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have",
                       "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'u.s':'america', 'e.g':'for example'}

In [9]:
def clean_text(text):
    #text = str(text).lower()    # Making Text Lowercase
    text = re.sub('b\"|b\'|\\\\|\\\"', '', str(text)) # remove the format that text starts with b" or b'
    text = re.sub('\[.*?\]', '', text) # remove text in [] brackets
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",", "'")
    text = re.sub(r"[^a-zA-Z?.!,¿']+", " ", text)
    return text

def clean_contractions(text, mapping):
    # Clean contraction by using contraction mapping   
    for word in mapping.keys():
        if str(word) in text:
            text = text.replace(str(word), str(mapping[word]))
    #Remove Punctuations
    text = re.sub('[^a-zA-Z]', ' ', text)
 
    return text

def remove_space(text): 
    ## remove space that are too long
    text = text.strip()
    text = text.split()
    return " ".join(text)

def text_preprocessing_pipeline(text):
    text = clean_text(text)
    text = clean_contractions(text, contraction_mapping)
    text = remove_space(text)
    return text

In [10]:
for i in range(df_train.shape[0]):
    for j in range(2, df_train.shape[1]):
        df_train.iloc[i,j] = text_preprocessing_pipeline(df_train.iloc[i,j])
df_train.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25,combined_news
0,2008/8/8,0,Georgia downs two Russian warplanes as countri...,BREAKING Musharraf to be impeached,Russia Today Columns of troops roll into South...,Russian tanks are moving towards the capital o...,Afghan children raped with impunity U N offici...,Russian tanks have entered South Ossetia whils...,Breaking Georgia invades South Ossetia Russia ...,The enemy combatent trials are nothing but a s...,...,Al Qaeda Faces Islamist Backlash,Condoleezza Rice The US would not act to preve...,This is a busy day The European Union has appr...,Georgia will withdraw soldiers from Iraq to he...,Why the Pentagon Thinks Attacking Iran is a Ba...,Caucasus in crisis Georgia invades South Ossetia,Indian shoe manufactory And again in a series ...,Visitors Suffering from Mental Illnesses Banne...,No Help for Mexico s Kidnapping Surge,Georgia downs two Russian warplanes as countri...
1,2008/8/11,1,Why wont America and Nato help us If they wont...,Bush puts foot down on Georgian conflict,Jewish Georgian minister Thanks to Israeli tra...,Georgian army flees in disarray as Russians ad...,Olympic opening ceremony fireworks faked,What were the Mossad with fraudulent New Zeala...,Russia angered by Israeli military sale to Geo...,An American citizen living in S Ossetia blames...,...,Do not believe TV neither Russian nor Georgian...,Riots are still going on in Montreal Canada be...,China to overtake US as largest manufacturer,War in South Ossetia,Israeli Physicians Group Condemns State Torture,Russia has just beaten the United States over ...,Perhaps the question about the Georgia Russia ...,Russia is so much better at war,So this is what it is come to trading sex for ...,Why wont America and Nato help us If they wont...
2,2008/8/12,0,Remember that adorable year old who sang at th...,Russia ends Georgia operation,If we had no sexual harassment we would have n...,Al Qa eda is losing support in Iraq because of...,Ceasefire in Georgia Putin Outmaneuvers the West,Why Microsoft and Intel tried to kill the XO l...,Stratfor The Russo Georgian War and the Balanc...,I am Trying to Get a Sense of This Whole Georg...,...,Why Russias response to Georgia was right,Gorbachev accuses U S of making a serious blun...,Russia Georgia and NATO Cold War Two,Remember that adorable year old who led your c...,War in Georgia The Israeli connection,All signs point to the US encouraging Georgia ...,Christopher King argues that the US and NATO a...,America The New Mexico,BBC NEWS Asia Pacific Extinction by man not cl...,Remember that adorable year old who sang at th...
3,2008/8/13,0,U S refuses Israel weapons to attack Iran report,When the president ordered to attack Tskhinval...,Israel clears troops who killed Reuters cameraman,Britain s policy of being tough on drugs is po...,Body of year old found in trunk Latest ransom ...,China has moved million quake survivors into p...,Bush announces Operation Get All Up In Russia ...,Russian forces sink Georgian ships,...,US humanitarian missions soon in Georgia if Ru...,Georgia s DDOS came from US sources,Russian convoy heads into Georgia violating truce,Israeli defence minister US against strike on ...,Gorbachev We Had No Choice,Witness Russian forces head towards Tbilisi in...,Quarter of Russians blame U S for conflict poll,Georgian president says US military will take ...,Nobel laureate Aleksander Solzhenitsyn accuses...,U S refuses Israel weapons to attack Iran repo...
4,2008/8/14,1,All the experts admit that we should legalise ...,War in South Osetia pictures made by a Russian...,Swedish wrestler Ara Abrahamian throws away me...,Russia exaggerated the death toll in South Oss...,Missile That Killed Inside Pakistan May Have B...,Rushdie Condemns Random House s Refusal to Pub...,Poland and US agree to missle defense deal Int...,Will the Russians conquer Tblisi Bet on it no ...,...,Georgia confict could set back Russia s 

In [11]:
for i in range(df_valid.shape[0]):
    for j in range(2, df_valid.shape[1]):
        df_valid.iloc[i,j] = text_preprocessing_pipeline(df_valid.iloc[i,j])
df_valid.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25,combined_news
0,2014/12/3,1,British man found with nailbomb weapons cache ...,Israel must give up Nuclear Weapons United Nat...,Ukraine PM discloses accident at nuclear plant...,Russian state TV s latest attempt to persuade ...,DNA Confirms King Richard III s Remains Under ...,An Indian state has halted mass sterilisations...,North Korea Bans Name Jong un for All but Lead...,President Bashar al Assad of Syria says US led...,...,Israel vents their fury with France after parl...,Indian forces kill militants crossing into Kas...,Melting Rate of West Antarctica Glaciers Tripl...,Green Energy Sector Jobs Surpass Total Oil San...,Jordanian Muslim Brotherhood smuggling weapons...,Questions raised over Queens ancestry after DN...,Topless Femen activist goes on trial in Cologn...,West Antarctic Melt Rate Has Tripled The total...,NASA has announced that a test launch of their...,British man found with nailbomb weapons cache ...
1,2014/12/4,0,NASA officially announces Manned Mars Mission ...,Vatican finds hundreds of millions of euros tu...,Major Cruise Lines Released Over a Billion Gal...,Isis recruits are begging to be allowed to ret...,Angela Merkel speaks against net neutrality,President Vladimir Putin warned Russians of ha...,Iran sentences blogger to death for insulting ...,ISIS claims constructing dirty bomb after stea...,...,Worlds Biggest Cave in Threatened by Cable Car...,South Korea Will Build A Giant Christmas Tree ...,U K Introduces the Google Tax The government t...,Cement Intended for Gaza Reconstruction Ends U...,A Japanese artist who made a kayak modelled on...,Putin Vows to Punish Speculators Pushing Down ...,NASA s Orion first test flight for Manned Mars...,Iran has provided evidence to the U N atomic a...,Life under ISIS Rule a Living Hell Mosul Iraq ...,NASA officially announces Manned Mars Mission ...
2,2014/12/5,1,Orion deep space capsule launches,Dozens of Chinese held in Kenya accused of pre...,Germany tries year old man who fought with ISI...,Blame your boss not China for stagnant wage gr...,France ordered to pay Somali pirates up to eac...,Chinese researchers report that a common pantr...,China overtakes the US as world s largest economy,Tony Blair says politicians are underpaid,...,Corruption a problem for all latest global ind...,Islamic Militants Attack Grozny Chechnyas Capi...,The new m Alan Turing Institute for Data Scien...,Cameroons National Assembly has voted into law...,U K to Repay World War I Loan The War Loan a s...,UK mass surveillance laws do not breach human ...,Latvian politician forced to step down after t...,Reports Obama Mulling Sanctions on Israel,Putin tells Russia Hard times are coming,Orion deep space capsule launches Dozens of Ch...
3,2014/12/8,0,Mass surveillance exposed by Snowden not justi...,The body temperature of Polish toddler rescued...,Mexico says evidence proves missing students w...,Sixteen British police officers have been char...,The best known liberal TV channel in Russia ha...,UBER banned in India s capital city New Delhi ...,New Horizons robotic probe awakened begins exp...,Uber the U S online ride hailing service has b...,...,EU parliament calls on Pakistan to abolish the...,Turkish President Erdogan says that the new pr...,In Canada More Jobs in Green Energy than Tar S...,Poland has long seen itself as stuck between t...,Rome Mayor Defies Mobster Threats and Refuses ...,German Chancellor Angela Merkel accuses Russia...,Arrested For Looting Antiquities From Israel s...,Ukraine to Allow Russian Electricity Imports A...,BC poultry farms have been placed under quaran...,Mass surveillance exposed by Snowden not justi...
4,2014/12/9,0,Hackers tell Sony to halt the release of The I...,Elephant Slaughter Ivory Sales Out of Control ...,Germany Thousands join anti Islam protest agai...,NSA warrantless bulk phone metadata spying con...,Hong Kong protesters given Thursday d

In [12]:

for i in range(df_test.shape[0]):
    for j in range(2, df_test.shape[1]):
        df_test.iloc[i,j] = text_preprocessing_pipeline(df_test.iloc[i,j])
df_test.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25,combined_news
0,2015/9/18,0,Brazil s Supreme Court has banned corporate co...,Investigation finds Exxon knew about CO s effe...,Islamic extremist shot dead in Berlin after st...,Man sentenced to death in Saudi Arabia will be...,Finland has a strike over people have gathered...,Cameron gives top environment policy job to oi...,India is not an IT superpower but an IT labour...,Bid to inspect Israeli nuclear site blocked,...,One million Yemenis ready to invade Saudi Arab...,Israeli police could use live sniper fire agai...,Japan passes new military bill Scuffling ensues,Russia says ready for talks with United States...,Farmers in northern France have been ordered t...,Super gonorrhoea outbreak in Leeds,In egalitarian Sweden richer regions reluctant...,China Is Building The Mother Of All Reputation...,Half a million children have fled attacks by t...,Brazil s Supreme Court has banned corporate co...
1,2015/9/21,1,Volkswagen has admitted to using software on V...,Saudi Teenager Ali Mohammed al Nimr Has Been S...,German woman is charged with counts of accesso...,New Zealand Zoo Won t Euthanize Tiger That Kil...,Hungary passes law allowing govt to use army i...,Hungary places ads in Lebanese press warning o...,Ruble fall causes wage parity between Russia a...,The Obama administration will increase the num...,...,ISIS Defectors Reveal Growing Disillusionment,Ministry of Defence condemns serving army gene...,Apple s App Store infected with malware in China,Islamic rebels free American hostages held in ...,Israel to bring in Chinese construction workers,Reports Russian troops refusing deployment to ...,At least people killed by three bomb blasts in...,Climate Change Will Boost Arctic Mosquito Popu...,Greece opposition concedes defeat Syriza poise...,Volkswagen has admitted to using software on V...
2,2015/9/22,0,Another drug Cycloserine sees a price jump ove...,Hedge fund trader Martin Shkreli who raised th...,American soldiers told to ignore sexual abuse ...,An engineer who once worked for Russia s milit...,VW emissions scandal hits m vehicles world wide,Israeli Police Can Now Use Snipers Against Tee...,Fewer than one in lobbyists are covered by the...,Seeds removed from doomsday seed vault as a re...,...,Facebook accused of spying on Belgian citizens...,German Stocks Fall More Than Pulled Down by Vo...,Midwife in the Indian state of Assam registere...,Exxon Confirmed Global Warming Consensus in wi...,Syrian asylum seeker in Germany filmed boastin...,Syrian refugee tripped over by Hungarian journ...,Land Degradation Desertification Might Create ...,European Union officials will forge ahead with...,German police conduct raids in Berlin s Islami...,Another drug Cycloserine sees a price jump ove...
3,2015/9/23,0,Pharmaceuticals CEO who raised HIV drug price ...,Norway rewards Brazil with billion for keeping...,Volkswagen hires BP oil spill lawyers to defen...,Turing to roll back pricing for Daraprim amid ...,Ginger extremist convicted in royal death plot...,Martin Winterkorn resigns as Volkswagen CEO,Pope Francis to Dine With Homeless Rather Than...,This year old won the Google Science Fair with...,...,Putin opens the biggest mosque in Russia prais...,World s largest D mud printer makes dirt cheap...,Hillary Clinton comes out against Keystone XL ...,The Israeli secret service is suspected of bei...,Canadians eating less meat taking a bite out o...,David Cameron says Europe must get better at s...,Eight Million Tons of Plastic Dumped in Ocean ...,A civil lawsuit against Volkswagen was filed T...,Diesel worse than the black plague and respons...,Pharmaceuticals CEO who raised HIV drug price ...
4,2015/9/24,0,Canadian mining company Barrick Gold leaks a m...,Nigerian troops free women children in Boko Ha...,Hajj dead and injured in Mina stampede,The Associate Press will no longer use the ter...,German commander in charge of anti ISIS

In [13]:
columns = ['Date', 'Top1', 'Top2','Top3', 'Top4', 'Top5','Top6', 'Top7', 'Top8', 'Top9', 'Top10', 'Top11', 'Top12', 'Top13', 'Top14', 'Top15', 'Top16', 'Top17', 'Top18', 'Top19', 'Top20', 'Top21', 'Top22', 'Top23','Top24', 'Top25']
df_train = df_train.drop(columns, axis=1)
df_train.head()

,Label,combined_news
0,0,Georgia downs two Russian warplanes as countri...
1,1,Why wont America and Nato help us If they wont...
2,0,Remember that adorable year old who sang at th...
3,0,U S refuses Israel weapons to attack Iran repo...
4,1,All the experts admit that we should legalise ...


In [14]:
columns = ['Date', 'Top1', 'Top2','Top3', 'Top4', 'Top5','Top6', 'Top7', 'Top8', 'Top9', 'Top10', 'Top11', 'Top12', 'Top13', 'Top14', 'Top15', 'Top16', 'Top17', 'Top18', 'Top19', 'Top20', 'Top21', 'Top22', 'Top23','Top24', 'Top25']
df_valid = df_valid.drop(columns, axis=1)
df_valid.head()

,Label,combined_news
0,1,British man found with nailbomb weapons cache ...
1,0,NASA officially announces Manned Mars Mission ...
2,1,Orion deep space capsule launches Dozens of Ch...
3,0,Mass surveillance exposed by Snowden not justi...
4,0,Hackers tell Sony to halt the release of The I...


In [15]:
columns = ['Date', 'Top1', 'Top2','Top3', 'Top4', 'Top5','Top6', 'Top7', 'Top8', 'Top9', 'Top10', 'Top11', 'Top12', 'Top13', 'Top14', 'Top15', 'Top16', 'Top17', 'Top18', 'Top19', 'Top20', 'Top21', 'Top22', 'Top23','Top24', 'Top25']
df_test = df_test.drop(columns, axis=1)
df_test.head()

,Label,combined_news
0,0,Brazil s Supreme Court has banned corporate co...
1,1,Volkswagen has admitted to using software on V...
2,0,Another drug Cycloserine sees a price jump ove...
3,0,Pharmaceuticals CEO who raised HIV drug price ...
4,0,Canadian mining company Barrick Gold leaks a m...


In [16]:
max_len = 0
for i in range(df_train.shape[0]):
  leng = len(df_train.iloc[i, 1].split())
  if max_len < leng:
    max_len = leng
print(max_len)

724


In [17]:
def stopwords_removal(news):
  word_list = news.split()
  tmp = []
  for word in word_list:
    if word.lower() not in stopwords.words('english'):
      tmp.append(word)
    filtered_news_article = ' '.join(tmp)
    return filtered_news_article

In [18]:
train_data = (df_train['combined_news'].to_list(), df_train['Label'].to_list())

In [19]:
valid_data = (df_valid['combined_news'].to_list(), df_valid['Label'].to_list())

In [20]:
test_data = (df_test['combined_news'].to_list(), df_test['Label'].to_list())

In [21]:
def create_dataset(texts: List[str],labels: List[int],num_classes: int = 2,class_names: List[int] = [0, 1]) -> Dataset:
    """
    Create HuggingFace Dataset from texts and labels.
    """
    ds = Dataset.from_dict({'text': texts, 'label': labels})
    ds = ds.cast_column('label', ClassLabel(num_classes=num_classes, names=class_names))
    return ds

In [23]:
def fine_tune(
    train_data: Tuple[List[str], List[int]], # text, label
    eval_data: Tuple[List[str], List[int]],
    model_output_dir: Union[str, Path],
    model_name: str = "distilbert-base-uncased",
    num_classes: int = 2,
    class_names: List[int] = [0, 1],
    learning_rate: float = 1e-5,
    weight_decay: float = 0.3,
    steps: int = 50,
    batch_size: int = 16,
    epochs: int = 20,
    verbose: bool = True
) -> Trainer:
    """
    Fine-tune a transformer for text classification.
    """

    def model_init():
        return AutoModelForSequenceClassification.from_pretrained(
            model_name, num_labels=num_classes, ignore_mismatched_sizes=True # new
        )
    
    # create datasets
    if verbose:
        print('Creating train and evaluation datasets.')
    train_ds = create_dataset(train_data[0], train_data[1], num_classes, class_names)
    eval_ds = create_dataset(eval_data[0], eval_data[1], num_classes, class_names)

    # perform shuffling
    train_ds = train_ds.shuffle(seed=42)
    eval_ds = eval_ds.shuffle(seed=42)

    if verbose:
        print(f'Tokenizing inputs.')
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenize = lambda eg: tokenizer(eg['text'], truncation=True)
    tokenized_train = train_ds.map(tokenize, batched=True)
    tokenized_eval = eval_ds.map(tokenize, batched=True)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    # define training arguments
    training_args = TrainingArguments(
        output_dir=model_output_dir,
        evaluation_strategy='steps',#steps or epoch
        # save_strategy = 'epoch',
        eval_steps=steps,
        logging_strategy='steps',
        logging_steps=steps,
        #saving_strategy='steps',
        #saving_steps=steps,
        save_strategy='steps',
        save_steps=steps,
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=epochs, 
        load_best_model_at_end=True,
    )

    # perform fine-tuning
    if verbose:
        print("Fine-tuning model.")
    trainer = Trainer(
        model_init=model_init, 
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_eval,
        tokenizer=tokenizer,
        data_collator=data_collator,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=5, early_stopping_threshold=0.0)]
    )
    trainer.train()
    # save model
    if verbose:
        print(f'Saving best model to {model_output_dir}/best-model.')
    trainer.save_model(f'{model_output_dir}/best-model')
    return trainer

In [24]:
def predict(trainer: Trainer, texts: List[str]) -> Tuple[List[int], List[float]]:
    """
    Return predicted class label and probabilities.
    """

    # create dataset and tokenize
    ds = Dataset.from_dict({'text': texts})
    tokenize = lambda eg: trainer.tokenizer(eg['text'], truncation=True, padding=True)
    tokenized_ds = ds.map(tokenize, batched=True)
    # make predictions
    logits = trainer.predict(tokenized_ds).predictions
    softmax = scipy.special.softmax(logits, axis=-1)
    prediction = softmax.argmax(axis=-1).tolist()
    probabilities = softmax.tolist()
    return prediction, probabilities

In [25]:
def evaluate(y: List[int], y_pred: List[int], class_names: List[int] = [0, 1], verbose=True):
    """
    Return evaluation metrics and print the confusion matrix.
    """

    # generate confusion matrix
    cf_mat = metrics.confusion_matrix(y, y_pred)
    row_names = ['Actual ' + str(c) for c in class_names]
    col_names = ['Predicted ' + str(c) for c in class_names]
    cf_mat = pd.DataFrame(cf_mat, index=row_names, columns=col_names)

    # calculate performance metrics
    accuracy = metrics.accuracy_score(y, y_pred)
    precision = metrics.precision_score(y, y_pred, average=None)
    recall = metrics.recall_score(y, y_pred, average=None)
    f1 = metrics.f1_score(y, y_pred, average=None)

    output_dict = {}
    output_dict['Accuracy'] = accuracy
    output_dict['Precision'] = precision
    output_dict['Recall'] = recall
    output_dict['F1'] = f1

    #print("Debug1:") ###
    if verbose:
        print(cf_mat)
        for k, v in output_dict.items():
            #print("Debug2:") ###
            print(k, ': ', v)
    
    return output_dict

In [26]:
def fine_tune_pipeline(
    train_data: Tuple[List[str], List[int]], # text, label
    eval_data: Tuple[List[str], List[int]],
    model_output_dir: Union[str, Path],
    model_name: str = "distilbert-base-uncased",
    num_classes: int = 2,
    class_names: List[int] = [0, 1],
    learning_rate: float = 1e-5,
    weight_decay: float = 0.3,
    steps: int = 50,
    batch_size: int = 16,
    epochs: int = 20,
    verbose: bool = True
) -> Dict[str, Any]:
    """
    Pipeline for fine-tuning a transformer and evaluating the performance.
    """

    # fine-tuning
    trainer = fine_tune(
        train_data,
        eval_data,
        model_output_dir,
        model_name,
        num_classes,
        class_names,
        learning_rate,
        weight_decay,
        steps,
        batch_size,
        epochs,
        verbose
    )

    # predict and evaluate on train & validation data
    train_pred, train_proba = predict(trainer, train_data[0])
    eval_pred, eval_proba = predict(trainer, eval_data[0])

    if verbose:
        print('Performance of tuned model on training set:')
    train_metrics = evaluate(train_data[1], train_pred, class_names)
    print()
    if verbose:
        print('Performance of tuned model on evaluation set:')
    eval_metrics = evaluate(eval_data[1], eval_pred, class_names)

    output = {
        'train_prediction': train_pred,
        'train_probabilities': train_proba,
        'train_performance': train_metrics,
        'eval_prediction': eval_pred,
        'eval_probabilities': eval_proba,
        'eval_performance': eval_metrics,
    }

    return output

In [27]:
def transformer_predict(
    sentences: List[str],
    model_dir: str,
    batch_size: int = None,
) -> Tuple[List[int], List[float]]:
    """
    Return predicted probabilities and labels.
    """

    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    # load tuned tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_dir)
    model = AutoModelForSequenceClassification.from_pretrained(model_dir)
    model = model.to(device)
    # make predictions
    preds_all, probas_all = [], []
    start_time = time.time()
    batch_size = batch_size if batch_size else len(sentences)
    num_examples = 0
    print("Making predictions...")
    with torch.no_grad():
        # process examples in batches
        for i in range(0, len(sentences), batch_size):
            if i <= len(sentences) - batch_size:
                batch = sentences[i : i + batch_size]
            else:
                batch = sentences[i:]
            num_examples += len(batch)
            # tokenize
            inputs = tokenizer(batch, truncation=True, padding=True, return_tensors="pt").to(device)
            # predict
            outputs = model(**inputs)
            # apply softmax
            softmax = torch.softmax(outputs.logits, dim=-1)
            probas = softmax.tolist()
            preds = softmax.argmax(dim=-1).tolist()
            preds_all.extend(preds)
            probas_all.extend(probas)
            # keep track of time taken
            now = time.time()
            duration = (now - start_time) / 60
            print(f"Predicted {num_examples} examples in {round(duration, 2)} minutes.")
    return preds_all, probas_all

In [28]:
save_model_dir = "/content/drive/MyDrive/FinBERT/save_weights"
output = fine_tune_pipeline(
    train_data, 
    valid_data, 
    model_output_dir=save_model_dir,
    model_name="ProsusAI/finbert",
    learning_rate=1e-5,
    weight_decay=0.1,
)

Creating train and evaluation datasets.


Casting the dataset:   0%|          | 0/1591 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/199 [00:00<?, ? examples/s]

Tokenizing inputs.


Map:   0%|          | 0/1591 [00:00<?, ? examples/s]

...

Step,Training Loss,Validation Loss
50,0.698100,0.699081
100,0.703000,0.700766
150,0.699200,0.704284
200,0.690100,0.694215
250,0.686400,0.732059
300,0.710700,0.713880
350,0.662100,0.736676
400,0.647500,0.732004
450,0.546000,0.823586


Saving best model to /content/drive/MyDrive/FinBERT/save_weights/best-model.


Map:   0%|          | 0/1591 [00:00<?, ? examples/s]

Map:   0%|          | 0/199 [00:00<?, ? examples/s]

Performance of tuned model on training set:
          Predicted 0  Predicted 1
Actual 0          682           46
Actual 1          691          172
Accuracy :  0.5367693274670019
Precision :  [0.49672251 0.78899083]
Recall :  [0.93681319 0.19930475]
F1 :  [0.64921466 0.31822387]

Performance of tuned model on evaluation set:
          Predicted 0  Predicted 1
Actual 0           98            6
Actual 1           94            1
Accuracy :  0.49748743718592964
Precision :  [0.51041667 0.14285714]
Recall :  [0.94230769 0.01052632]
F1 :  [0.66216216 0.01960784]
